<a href="https://colab.research.google.com/github/TristanConant/sales-predictions/blob/final/ml_sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Data

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [50]:
file_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQcO5VAKyttMX8k6NqLE5Q5wHBt1ZVvuQ-Emy8aAvUOlbLrt_dcvqbBnGLtI3fDP_gAgdlmlfed1c3i/pub?gid=883441261&single=true&output=csv'
df = pd.read_csv(file_url)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


Before anything else, I will make a copy to preserve the integrity of the original dataframe. Then using the info method on the dataframe I will get an overview of the columns, rows, and column types.

In [51]:
ml_df = df.copy()
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


# DataFrame Management and Cleanup



In [52]:
# Checking for duplicates
ml_df.duplicated().sum()

0

In [53]:
# Checking for missing data.
# Missing values will be handled during column transformation
ml_df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [54]:
# Loop over object columns and return a count of  unique values
dtypes = ml_df.dtypes
str_cols = dtypes[dtypes=='object'].index

for col in str_cols:
  print(f'Column: {col}')
  print(ml_df[col].value_counts())
  print('\n\n')

Column: Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64



Column: Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64



Column: Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



Column: Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT01

In [55]:
# Fixing object column inconsistencies
ml_df['Item_Fat_Content'].replace('low fat', 'Low Fat', inplace=True)
ml_df['Item_Fat_Content'].replace('LF', 'Low Fat', inplace=True)
ml_df['Item_Fat_Content'].replace('reg', 'Regular', inplace=True)

# Checking that the changes were applied as expected
for col in str_cols:
  print(f'Column: {col}')
  print(ml_df[col].value_counts())
  print('\n\n')

Column: Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64



Column: Item_Fat_Content
Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64



Column: Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



Column: Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    5

# Preprocessing

#### Train test split

In [56]:
# Defining the feature set by dropping the target column and irrelevant columns. 
X = ml_df.drop(columns=['Item_Outlet_Sales', 'Item_Identifier', 'Outlet_Establishment_Year'])
# Defining the target set.
y = ml_df['Item_Outlet_Sales']

In [57]:
# Performing the train test split.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#### Preprocessing pipelines


In [58]:
# Instantiating standard scaler
scaler = StandardScaler()

# Instantiating OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')

# Instantiating simple imputers
mean_imputer = SimpleImputer(strategy='mean')
most_frequent_imputer = SimpleImputer(strategy='most_frequent')

# Defining list of values for ordinal encoder
size_labels = ['High', 'Medium', 'Small']
type_labels = ['Tier 1','Tier 2','Tier 3']
ordered_labels = [size_labels, type_labels]

# Instantiating ordinal encoder
ordinal = OrdinalEncoder(categories=ordered_labels)

In [59]:
# Numeric pipline
num_pipe = make_pipeline(mean_imputer, scaler)
num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [60]:
# Ordinal pipeline
ord_pipe = make_pipeline(most_frequent_imputer, ordinal, scaler)
ord_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['High', 'Medium', 'Small'],
                                            ['Tier 1', 'Tier 2', 'Tier 3']])),
                ('standardscaler', StandardScaler())])

In [61]:
# Nominal pipline
nom_pipe = make_pipeline(most_frequent_imputer, ohe)
nom_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

### Column transformation

In [62]:
# Creating column selectors
num_selector = make_column_selector(dtype_include='number')
cat_selector = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Type']
ord_selector = ['Outlet_Size', 'Outlet_Location_Type']

# Matching transformers with columns
num_tuple = (num_pipe, num_selector)
nom_tuple = (nom_pipe, cat_selector)
ord_tuple = (ord_pipe, ord_selector)

In [63]:
# Instantiate the ColumnTransformer
col_transformer = make_column_transformer(num_tuple, nom_tuple, ord_tuple, remainder='passthrough')

In [64]:
# Fit transformer to the train data
col_transformer.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fc48c2ae890>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHo...ore'))]),
                                 ['Item_Fat_Content', 'Item_Type',
                                  'Outlet_Identifier', 'Outlet_Type']),
                                ('pip

In [65]:
# Transform the training and test data sets
X_train = col_transformer.transform(X_train)
X_test = col_transformer.transform(X_test)

# Linear Regression Model

In [66]:
# Instantiating linear regression model
reg = LinearRegression()

In [67]:
# Fitting the model to the train data
reg.fit(X_train, y_train)

LinearRegression()

### Regression model metrics

In [69]:
# Storing model predictions on the test data
train_pred = reg.predict(X_train)
test_pred = reg.predict(X_test)

# R2
train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)